In [65]:
import numpy as np
from equations import *
from equations_matrix import *
from solvers import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Test comparing numerical and analytical solution for X_prme

### 1.1 parameter setup

In [ ]:
# Set dimensions
N = 51  # number of countries
J = 3  # number of sectors

# Generate dummy arrays for the exogenous inputs
w_hat = np.ones(N)
td_prime = np.zeros(N)

# pif_hat and pim_hat are (N, N, J)
pif_hat = np.ones((N, N, J)) / N
pim_hat = np.ones((N, N, J)) / N


# Define dummy classes for ModelParams and ModelShocks
class ModelParams:
    pass

class ModelShocks:
    pass

# Create instances
mp = ModelParams()
shocks = ModelShocks()

# Fill ModelParams with attributes
mp.w0 = np.ones(N)
mp.L0 = np.ones(N)
mp.alpha = np.ones((N, J)) / J
mp.pif = np.ones((N, N, J))
mp.pim = np.ones((N, N, J)) 
mp.gamma = np.ones((N, J, J)) * 0.7 / J

# Fill ModelShocks with attributes
# Ensure tilde_tau_prime > 1 to avoid division by zero issues
shocks.tilde_tau_prime = np.random.rand(N, N, J) + 1.1

### 1.2 Numerical solution

In [15]:
Xf_init = mp.alpha * (w_hat * mp.w0 * mp.L0 + td_prime)[:, None]  # shape: (N, J)
Xm_init = np.zeros_like(Xf_init)                                   # shape: (N, J)

# Call your iterative solver (solve_X_prime) using the guess:
Xf_iter, Xm_iter = solve_X_prime(w_hat, pif_hat, pim_hat, td_prime, Xf_init, Xm_init, mp, shocks, mute=False)

X_prime converged in 57 iterations


### 1.3 Analytical solution

In [16]:
# Call the closed-form solver (calc_X) to get the direct solution:
Xf_closed, Xm_closed = calc_X(w_hat, pif_hat, pim_hat, td_prime, mp, shocks)

### 1.4 Check the difference

In [17]:
# Compare the results between the iterative and closed-form solutions.
print("Difference in Xf (iterative vs. closed-form):", np.linalg.norm(Xf_iter - Xf_closed))
print("Difference in Xm (iterative vs. closed-form):", np.linalg.norm(Xm_iter - Xm_closed))

Difference in Xf (iterative vs. closed-form): 4.2999443126792367e-05
Difference in Xm (iterative vs. closed-form): 5.4974185203698866e-05


# 2. HHI 

### Mathematical Format

Possibly wrong:

$$Q_n^s \equiv \frac{\sum_{i}(\pi_{in}^{sf}X_{in}^{sf}+\pi_{in}^{sm}X_{in}^{sm})}{1+\tau_{in}}$$

Inferred:

$$Q_n^s \equiv \sum_{i}\frac{(\pi_{in}^{sf}X_{i}^{sf}+\pi_{in}^{sm}X_{i}^{sm})}{1+\tau^s_{in}}$$
Then,HHI can be calculated as
$$HHI_n=\sum_k\left(\frac{Q_n^k}{\sum_s Q_n^s}\right)^2$$

In [66]:
def calc_HHI(pim, pif, Xm, Xf, tau_tilde):
    """
    - pif, pim, tau_tilde: arrays of shape (i, n, s)
        where:
            i = importing country index,
            n = exporting country index,
            s = sector index.
    - Xm, Xf: arrays of shape (i, s) corresponding to intermediate and final goods expenditures
        for importing countries.
    The formula is:
      Q_n^s = sum_{i} [ (pif[i,n,s] * Xf[i,s] + pim[i,n,s] * Xm[i,s]) / (1 + tau_tilde[i,n,s]) ]
    and then the HHI for each exporting country n is:
      HHI_n = sum_s ( Q_n^s / (sum_{s'} Q_n^{s'}) )^2.
    
    Returns:
      HHI: an array of shape (n,) giving the HHI for each exporting country.
    """
    numerator = pif * Xf[:, None, :] + pim * Xm[:, None, :]  # shape: (N, N, J)
    term = numerator / (1 + tau_tilde)  # shape: (N, N, J)
    Q = term.sum(axis=0)  # shape: (N, J)
    # For each exporting country, sum over sectors to get the total Q:
    Q_total = Q.sum(axis=1)  # shape: (N,)
    # Compute HHI for each exporting country:
    HHI = np.sum((Q / Q_total[:, None])**2, axis=1)  # shape: (N,)
    return HHI

# 3. Test to see if this paper generalize to CP(2015)

### 3.1 parameter setup (same as 1.1)

In [72]:
# Set dimensions
N = 51  # number of countries
J = 3  # number of sectors

# Generate dummy arrays for the exogenous inputs
w_hat = np.ones(N)
td_prime = np.zeros(N)

# pif_hat and pim_hat are (N, N, J)
pif_hat = np.ones((N, N, J)) / N
pim_hat = np.ones((N, N, J)) / N


# Define dummy classes for ModelParams and ModelShocks
class ModelParams:
    pass
class ModelShocks:
    pass
# Create instances
mp = ModelParams()
shocks = ModelShocks()

# Fill ModelParams with attributes
mp.w0 = np.ones(N)
mp.L0 = np.ones(N)
mp.alpha = np.ones((N, J)) / J
mp.pif = np.ones((N, N, J))
mp.pim = np.ones((N, N, J)) 
mp.gamma = np.ones((N, J, J)) * 0.7 / J

# Fill ModelShocks with attributes
# Ensure tilde_tau_prime > 1 to avoid division by zero issues
shocks.tilde_tau_prime = np.random.rand(N, N, J) + 1.1


### 3.2 Analytical solutions for $X_f, X_m$ in this paper

In [73]:
# Call the closed-form solver (calc_X) to get the direct solution:
Xf_closed, Xm_closed = calc_X(w_hat, pif_hat, pim_hat, td_prime, mp, shocks)

### 3.3 Analytical solution for $X$ in CP(2015)

In [74]:
from cp_expenditure_equation import calc_X_cp
# mapping between inputs for calc_X function and for calc_X_cp function
alpha = mp.alpha
gamma = mp.gamma
pi_prime = pif_hat * mp.pif
tilde_tau_prime = shocks.tilde_tau_prime
D = td_prime
VA = mp.w0 * mp.L0

X_cp = calc_X_cp(w_hat, alpha, gamma, pi_prime, tilde_tau_prime, D, VA)

### 3.4 Check if $X_f + X_m = X_{cp}$

In [80]:
assert (Xf_closed + Xm_closed == X_cp).all()